In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
from src.databases import Postgresql 
from imblearn.over_sampling import SMOTE
from sklearn.pipeline import make_pipeline
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.feature_selection import SelectKBest, chi2, f_classif
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression


import warnings
warnings.filterwarnings("ignore")
pd.set_option('max_columns', None)

In [3]:
# trazer o dataset com as viariaveis dummy do banco de dados
query = """ SELECT * FROM dataset_bf_encoder;"""
bd = Postgresql(user='postgres' , password='bruno22#' , host= 'localhost', port= '5432', database = 'brunods')
dataset= bd.retrieve_data(query=query)
dataset.drop_duplicates(inplace=True)
dataset.set_index('sku', inplace=True)
#dataset = dataset.astype('int')

Buscando os dados!!!
Conexão com Postgresql fechada


In [4]:
# Dando uma olhada no dataset
dataset.head()

,tipo,country,region,vinicola,classificacao,safra,temperatura,potencial_guarda,cat_safra,teor_cat,barris_carvalho,inox,metodo_tradicional,garrafas,concreto,gustativo,visual,ofativo,comments_label,uva,rating
sku,,,,,,,,,,,,,,,,,,,,,
11549,Branco,Hungria,Tokaj,Pajzos,Suave/Doce,2003,media_temperatura,vinho_guarda,None,teor_baixo,true,false,false,false,false,Acidez,Amarelo,outros,0,Furmint,0
12390,Branco,França,Languedoc-Roussillon,Baron Philippe de Rothschild,Seco,2018,media_temperatura,consumo_rapido,None,teor_baixo,false,true,false,false,false,Acidez,Amarelo,outros,0,Sauvignon Blanc,0
12733,Tinto,Espanha,Ribera del Júcar,La Magdalena,Seco,2017,alta_temperatura,consumo_rapido,None,teor_alto,true,false,false,false,false,outros,Rubi,outros,0,Tempranillo,0
12734,Branco,Chile,Valle Central,Casas del Toqui,Seco,2019,baixa_temperatura,consumo_rapido,None,teor_medio,false,true,false,false,false,Acidez,Amarelo,outros,0,Chardonnay,0
12813,Tinto,França,Pomerol,Château Gazin,Seco,2010,alta_temperatura,vinho_guarda,None,teor_alto,true,false,false,false,false,outros,Rubi,aroma_madeirado,0,Merlot,0


In [7]:
# checando o shape
dataset.shape

(453, 21)

In [8]:
# one-hot encoding - country, region, vinicola
data_temp = dataset.loc[dataset['rating']=='1',:].groupby('country').count()
data_temp.loc[:,'rating'] # fazer one-hot dos paises com mais de 50 rating 

country
Argentina         3
Austrália         2
Brasil            2
Chile             9
Espanha           5
Estados Unidos    2
França            5
Itália            5
Portugal          1
Name: rating, dtype: int64

In [9]:
# one-hot-encoding - region - com + de 10 (media+1desvp) 3 maiores

data_temp1 = dataset.loc[dataset['rating']=='1',:].groupby('region').count()
data_region = data_temp1.loc[:,'rating']
sample = (data_region > 10)
data_region[sample]

Series([], Name: rating, dtype: int64)

In [10]:
# one-hot-encoding - vinicola - com + de 10 (media+1desvp) 3 maiores iguais as regions 

data_temp2 = dataset.loc[dataset['rating']=='1',:].groupby('vinicola').count()
data_vinicola = data_temp1.loc[:,'rating']
sample = (data_vinicola > 5)
data_vinicola[sample]

Series([], Name: rating, dtype: int64)

In [11]:
# check para one hot encoding - variavel uva - 

data_temp3 = dataset.loc[dataset['rating']=='1',:].groupby('uva').count()
data_uva = data_temp3.loc[:,'rating']
sample = (data_uva >10)
data_uva[sample]

Series([], Name: rating, dtype: int64)

In [12]:
#One-Hot de variavel country - frança, Espanha, Chile, 
def one_hot_franca(x):
    if x == "França":
        return 1
    else:
        return 0

def one_hot_espanha(x):
    if x == "Espanha":
        return 1
    else:
        return 0   
    
def one_hot_chile(x):
    if x == "Chile":
        return 1
    else:
        return 0   
    
dataset['franca'] = dataset.country.apply(one_hot_franca)
dataset['espanha'] = dataset.country.apply(one_hot_espanha)
dataset['chile'] = dataset.country.apply(one_hot_chile)
dataset.drop('country', axis=1, inplace=True)

In [13]:
#One-Hot de variavel region - Valle Central - Mendoza - California

def one_hot_valle(x):
    if x == "Valle Central":
        return 1
    else:
        return 0

def one_hot_serra_gaucha(x):
    if x == "Serra Gaúcha":
        return 1
    else:
        return 0
    
def one_hot_mendoza(x):
    if x == "Mendoza":
        return 1
    else:
        return 0 

dataset['valle'] = dataset.region.apply(one_hot_valle)
dataset['serra_gaucha'] = dataset.region.apply(one_hot_serra_gaucha)
dataset['mendoza'] = dataset.region.apply(one_hot_mendoza)
dataset.drop('region', axis=1, inplace=True)

In [14]:
#One-Hot de variavel vinicola - valle_maipo, languedoc, tejo

def one_hot_valle_maipo(x):
    if x == "Valle del Maipo":
        return 1
    else:
        return 0

def one_hot_languedoc(x):
    if x == "Languedoc-Roussillon":
        return 1
    else:
        return 0
    
def one_hot_tejo(x):
    if x == "Tejo":
        return 1
    else:
        return 0

dataset['Valle_del_Maipo'] = dataset.vinicola.apply(one_hot_valle_maipo)
dataset['languedoc'] = dataset.vinicola.apply(one_hot_languedoc)
dataset['tejo'] = dataset.vinicola.apply(one_hot_tejo)
dataset.drop('vinicola', axis=1, inplace=True)

#Dropando 'safra' pq ja temos cat_safra
dataset.drop('safra', axis=1, inplace=True)

In [15]:
#One-hot-Encoding Variavel uva

def one_hot_cabernet_sauvignon(x):
    if x == 'Cabernet Sauvignon':
        return 1
    else:
        return 0

def one_hot_merlot(x):
    if x == 'Merlot':
        return 1
    else:
        return 0

def one_hot_chadornay(x):
    if x == 'Chardonnay':
        return 1
    else:
        return 0

dataset['Cabernet_Sauvignon'] = dataset.uva.apply(one_hot_cabernet_sauvignon)
dataset['Merlot'] = dataset.uva.apply(one_hot_merlot)
dataset['Chardonnay'] = dataset.uva.apply(one_hot_chadornay)
dataset.drop('uva', axis=1, inplace=True)

In [16]:
# Lable Encoding da demais variavies
le = LabelEncoder()

for i in dataset.columns.values:
    dataset[i] = le.fit_transform(dataset[i])

In [17]:
dataset.head()

,tipo,classificacao,temperatura,potencial_guarda,cat_safra,teor_cat,barris_carvalho,inox,metodo_tradicional,garrafas,concreto,gustativo,visual,ofativo,comments_label,rating,franca,espanha,chile,valle,serra_gaucha,mendoza,Valle_del_Maipo,languedoc,tejo,Cabernet_Sauvignon,Merlot,Chardonnay
sku,,,,,,,,,,,,,,,,,,,,,,,,,,,,
11549,0,3,2,1,0,1,1,0,0,0,0,0,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0
12390,0,2,2,0,0,1,0,1,0,0,0,0,0,4,0,0,1,0,0,0,0,0,0,0,0,0,0,0
12733,4,2,0,0,0,0,1,0,0,0,0,5,2,4,0,0,0,1,0,0,0,0,0,0,0,0,0,0
12734,0,2,1,0,0,2,0,1,0,0,0,0,0,4,0,0,0,0,1,0,0,0,0,0,0,0,0,0
12813,4,2,0,1,0,0,1,0,0,0,0,5,2,2,0,0,1,0,0,0,0,0,0,0,0,0,0,0


In [42]:
dataset = dataset.reset_index()

In [22]:

query = """ CREATE TABLE dataset_finalv2 ( sku TEXT,
                                            tipo                  int,
                                            classificacao         int,
                                            temperatura           int,
                                            potencial_guarda      int,
                                            cat_safra             int,
                                            teor_cat              int,
                                            barris_carvalho       int,
                                            inox                  int,
                                            metodo_tradicional    int,
                                            garrafas              int,
                                            concreto              int,
                                            gustativo             int,
                                            visual                int,
                                            ofativo               int,
                                            comments_label         int,
                                            rating                int,
                                            franca                int,
                                            espanha               int,
                                            chile                 int,
                                            valle                 int,
                                            serra_gaucha          int,
                                            mendoza               int,
                                            Valle_del_Maipo       int,
                                            languedoc             int,
                                            tejo                  int,
                                            Cabernet_Sauvignon    int,
                                            Merlot                int,
                                            Chardonnay            int)  """



bd = Postgresql(user='postgres' , password='bruno22#' , host= 'localhost', port= '5432', database = 'brunods')
bd.create_table(query=query)
bd.insert_data(df = dataset, tabela = ' dataset_finalv2')

Tabela criada no postgresql com sucesso 
Conexão com Postgresql fechada
Conexão com Postgresql fechada


In [43]:
#divisão dos dados
dataset.set_index('sku', inplace=True)
X= dataset.drop('rating', axis=1)
Y= dataset['rating']

#Divisão entre treino e teste
X_treino, X_teste, y_treino, y_teste = train_test_split(X,Y,test_size = 0.2, random_state=0)

# Balanciando o dataset
x, y = SMOTE().fit_resample(X_treino, y_treino)
print('Balanciamento realizado: ', sorted(Counter(y).items()))

Balanciamento realizado:  [(0, 336), (1, 336)]


In [25]:
# Testando chi2 ou f_classif

def chi2_anova(tec1, tec2):
    tecnicas = [tec1, tec2]

    for item in tecnicas:
        model_select = make_pipeline(
                    SelectKBest(item, k=5), ExtraTreesClassifier()
            )
        model_select.fit(x, y)
        print('Acuracia com {}: {:.3f}'.format(item, model_select.score(X_teste, y_teste)))

chi2_anova(chi2, f_classif)

Acuracia com <function chi2 at 0x7eff2ecba310>: 0.648
Acuracia com <function f_classif at 0x7eff2eca5f70>: 0.582


In [26]:
#RandomForestClassifier
def n_k_et(method, ks, model):
    num_k= ks
    for item in num_k:

        model_select = make_pipeline(
                SelectKBest(method, k=item),model(random_state=1)
        )
        model_select.fit(x, y)
        print('Acuracia com a quantidade k de {}: {:.3f}'.format(item, model_select.score(X_teste, y_teste)))

n_k_et(f_classif, ks=list(range(1,21)), model=RandomForestClassifier)

Acuracia com a quantidade k de 1: 0.286
Acuracia com a quantidade k de 2: 0.407
Acuracia com a quantidade k de 3: 0.495
Acuracia com a quantidade k de 4: 0.582
Acuracia com a quantidade k de 5: 0.582
Acuracia com a quantidade k de 6: 0.648
Acuracia com a quantidade k de 7: 0.648
Acuracia com a quantidade k de 8: 0.648
Acuracia com a quantidade k de 9: 0.703
Acuracia com a quantidade k de 10: 0.703
Acuracia com a quantidade k de 11: 0.659
Acuracia com a quantidade k de 12: 0.670
Acuracia com a quantidade k de 13: 0.703
Acuracia com a quantidade k de 14: 0.714
Acuracia com a quantidade k de 15: 0.725
Acuracia com a quantidade k de 16: 0.736
Acuracia com a quantidade k de 17: 0.758
Acuracia com a quantidade k de 18: 0.758
Acuracia com a quantidade k de 19: 0.747
Acuracia com a quantidade k de 20: 0.769


In [28]:
# chi2 by -Foward elimination best 16
k_best = SelectKBest(score_func=f_classif, k=17)
selected  = k_best.fit(x,y)
index = selected.get_support(indices=True)

print(f'Indices da variavies são : {index}')

#features
dados = dataset.iloc[:,index]

print('\n')
dados.head()

Indices da variavies são : [ 0  1  2  5  7  9 10 11 12 13 15 16 17 20 24 25 26]




,tipo,classificacao,temperatura,teor_cat,inox,garrafas,concreto,gustativo,visual,ofativo,rating,franca,espanha,serra_gaucha,tejo,Cabernet_Sauvignon,Merlot
sku,,,,,,,,,,,,,,,,,
11549,0,3,2,1,0,0,0,0,0,4,0,0,0,0,0,0,0
12390,0,2,2,1,1,0,0,0,0,4,0,1,0,0,0,0,0
12733,4,2,0,0,0,0,0,5,2,4,0,0,1,0,0,0,0
12734,0,2,1,2,1,0,0,0,0,4,0,0,0,0,0,0,0
12813,4,2,0,0,0,0,0,5,2,2,0,1,0,0,0,0,0


# Treinamento do Modelo

In [29]:
def random_RF(X,y,n_iter):
    
    print('> Procurando os melhores parametros...')
    n_estimators = [int(x) for x in np.linspace(start = 500, stop = 2000, num = 10)]
    max_features = ['auto', 'sqrt','log2']
    max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
    max_depth.append(None)
    criterion = ['gini','entropy']
    min_samples_split = [2, 5, 10, 12]
    min_samples_leaf = [1, 2, 4, 6, 8]
    bootstrap = [True, False]
    
    random_grid = {'max_features':max_features,
                   'criterion': criterion,
                   'bootstrap': bootstrap}
    
    random_state = 2
    rfc = RandomForestClassifier(n_jobs=-1)
    rf_random = RandomizedSearchCV(estimator=rfc, param_distributions=random_grid, 
                                   n_iter=n_iter, cv=5, verbose=0, random_state=random_state, 
                                   n_jobs=-1, scoring={'AUC':'roc_auc'},refit='AUC')

    # Fit the random search model
    print('> Fitting Modelo...')
    rf_random.fit(X, y)
  
    return rf_random.best_estimator_, rf_random.cv_results_, rf_random.best_params_

In [30]:
trained_model, results, params=random_RF(x,y,10)

> Procurando os melhores parametros...
> Fitting Modelo...


In [31]:
params

{'max_features': 'auto', 'criterion': 'entropy', 'bootstrap': True}

In [45]:
#classification report

pred = trained_model.predict(X_teste)
print(classification_report(pred,y_teste))

              precision    recall  f1-score   support

           0       1.00      0.91      0.95        91
           1       0.00      0.00      0.00         0

    accuracy                           0.91        91
   macro avg       0.50      0.46      0.48        91
weighted avg       1.00      0.91      0.95        91



In [25]:
query = """ SELECT * FROM dataset_finalv1;"""
bd = Postgresql(user='postgres' , password='bruno22#' , host= 'localhost', port= '5432', database = 'brunods')
teste= bd.retrieve_data(query=query)

Buscando os dados!!!
Conexão com Postgresql fechada


In [28]:
datasetv1.head()

,tipo,classificacao,temperatura,potencial_guarda,cat_safra,teor_cat,barris_carvalho,inox,metodo_tradicional,garrafas,...,chile,valle,california,mendoza,valle_del_maipo,languedoc,tejo,cabernet_sauvignon,merlot,chardonnay
sku,,,,,,,,,,,,,,,,,,,,,
11549,0,3,2,1,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12813,4,2,0,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12815,4,2,0,1,0,2,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12816,4,2,0,1,0,2,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12877,4,2,0,1,0,2,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Processo Total

In [50]:
# Processo de treinamento.

query = """ SELECT * FROM dataset_finalv2;"""
bd = Postgresql(user='postgres' , password='bruno22#' , host= 'localhost', port= '5432', database = 'brunods')
datasetv1= bd.retrieve_data(query=query)
datasetv1.set_index('sku', inplace=True)
            

#divisão dos dados
X= datasetv1.drop('rating', axis=1)
Y= datasetv1['rating']


k_best = SelectKBest(score_func=f_classif, k=15)
selected  = k_best.fit(X,Y)
index = selected.get_support(indices=True)


X1 = dados = datasetv1.iloc[:,index]
Y1 = Y


#Divisão entre treino e teste
X_train, X_test, y_train, y_test = train_test_split(X1,Y1,test_size = 0.2, random_state=0)

# Balanciando o dataset
x1, y1 = SMOTE().fit_resample(X_train, y_train)
print('Balanciamento realizado: ', sorted(Counter(y1).items()))
    
#trained_model, results, params=random_RF(x1,y1,10)

trained_model = LogisticRegression()
    
resultado = []
kf = RepeatedKFold(n_splits = 2, n_repeats=10, random_state=1)

for linhas_treino, linhas_valid in kf.split(x1):

    X_treino, X_valid = x1.iloc[linhas_treino], x1.iloc[linhas_valid]
    y_treino, y_valid = y1.iloc[linhas_treino], y1.iloc[linhas_valid]


    trained_model.fit(X_treino,y_treino)
    p= trained_model.predict(X_valid)
    acc = np.mean(y_valid==p)
    resultado.append(acc)
print(f'ROCAUC com crossvalidação de {roc_auc_score(y_valid,p)}')

#Em dados de teste
trained_model.fit(x1,y1)
p1 = trained_model.predict(X_test)
print(f'ROCAUC em dados de teste de {roc_auc_score(y_test,p1)}')

Buscando os dados!!!
Conexão com Postgresql fechada
Balanciamento realizado:  [(0, 336), (1, 336)]
ROCAUC com crossvalidação de 1.0
ROCAUC em dados de teste de 1.0


In [49]:
x1.head()

,tipo,classificacao,temperatura,potencial_guarda,teor_cat,barris_carvalho,inox,metodo_tradicional,garrafas,concreto,gustativo,visual,ofativo,rating,franca,espanha,valle,serra_gaucha,tejo,cabernet_sauvignon,merlot
0,4,2,0,1,2,1,0,0,0,0,0,2,4,0,1,0,0,0,0,0,0
1,4,2,0,0,2,0,1,0,0,0,5,2,4,0,1,0,0,0,0,0,0
2,3,2,1,0,1,0,1,0,0,0,0,1,4,0,0,0,0,0,0,0,1
3,0,2,2,1,2,1,0,0,0,0,0,0,4,0,1,0,0,0,0,0,0
4,4,1,0,1,2,1,0,0,0,0,0,2,4,0,0,0,0,0,0,0,0
